Use Pymatgen to generate some standard VASP input sets. 

This can be particularly useful if you have a high level of heterogeneity in a set of input structures (e.g. many different structure types or compositions). 

In [2]:
### Imports
import os
import json
from tqdm import tqdm
from pymatgen import Structure

from pymatgen.io.vasp.sets import DictSet

In [10]:
# Load in the structures
with open('data/example_input_structures.json', 'r') as f:
    structures = json.load(f)

# Convert back to pymatgen structure objects
for i in structures:
    i['structure'] = Structure.from_dict(i['structure'])
    
# Load the calculation config we will use
with open('data/calc_config_example.json', 'r') as f:
    calc_config = json.load(f)

We can see what the calculation config dict needs to look like. There is also a POTCAR dict which details which POTCAR to use for each element, should it appear in one of our structures. KPOINTS is defined as a reciprocal density. See the Pymatgen documentation for the many ways in which KPOINTS can be specified. 

In [16]:
print(calc_config.keys())
print(calc_config['INCAR'])
print(calc_config['KPOINTS'])

dict_keys(['INCAR', 'KPOINTS', 'POTCAR'])
{'ALGO': 'NORMAL', 'EDIFF': 1e-06, 'EDIFFG': -0.005, 'ENCUT': 600, 'IBRION': 2, 'ICHARG': 1, 'ISIF': 3, 'ISMEAR': 0, 'ISPIN': 2, 'LORBIT': 10, 'LREAL': 'AUTO', 'LWAVE': False, 'NELM': 100, 'NSW': 99, 'PREC': 'Accurate', 'SIGMA': 0.1, 'NCORE': 16, 'KPAR': 4}
{'reciprocal_density': 120}


Then we simply distribute the necessary files:

In [19]:
for n, i in enumerate(structures):
    dirname = 'data/example_calcs/{0}_{1}'.format(n, i['structure'].composition.reduced_formula)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    
    # Put candidate VASP files into dir
    v  = DictSet(i['structure'], config_dict = calc_config)
    v.write_input(dirname)

Next steps: Copy the entire `dirname` directory to a HPC cluster and run the calcs. 